# Context

One may filter different features in the neural signals. Here it is investigated which preprocessing steps are suitable in this respect.

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow import HornSchunck, getimgfiles
from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
np.array(np.clip([300],0,255), dtype=np.uint8)

### Import our custom utility methods

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

# Load data

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

In [ ]:
frames = skimage.io.imread(os.path.join(source_folder,"runstart16_X1.tif"))

In [ ]:
print(np.max(frames))

In [ ]:
print(np.min(frames))

In [ ]:
frames = frames[:1000,:,:]

# Preprocessing

Here I calculate the difference from pixelwise mean as well as a smoothed version that promised to increase the signal to noise ratio.

In [ ]:
mean = np.mean(frames,axis=0)#pixelwise mean

In [ ]:
np.mean(frames,axis=(1,2)).shape

In [ ]:
difference = framewise_difference(frames, mean, bigdata=True)
print(np.max(difference))
print(np.min(difference))
#difference = normalize(difference)
difference = difference/65535
smooth = gaussian_filter(difference, 2)

In [ ]:
plt.imshow(smooth[0])
plt.colorbar()

In [ ]:
%%capture
ani = show_video(smooth)

In [ ]:
HTML(ani)

In [ ]:
upper_decentile = [np.quantile(f,0.9) for f in smooth]
plt.plot(upper_decentile)

In [ ]:
smooth = remove_frequency_from_pixel_vectors(smooth,15,20)

In [ ]:
upper_decentile = [np.quantile(f,0.9) for f in smooth]
plt.plot(upper_decentile)

In [ ]:
smooth, explained_variance_per_component = discard_minor_components(smooth, 3)

In [ ]:
upper_decentile = [np.quantile(f,0.9) for f in smooth]
plt.plot(upper_decentile)

In [ ]:
plt.plot(explained_variance_per_component)

In [ ]:
%%capture
ani = show_video(smooth[70:170], n_frames=60)

In [ ]:
HTML(ani)

In [ ]:
from scipy.ndimage.morphology import grey_closing

In [ ]:
plt.imshow(smooth[0])

In [ ]:
closed = grey_closing(smooth, size=(0,30,10))

In [ ]:
diff = gaussian_filter(np.mean(np.abs(closed[0:-2]-closed[1:-1]), axis=(1,2)),5)
plt.plot(diff)


pois = []
#pois.extend(argrelextrema(diff,np.greater)[0])
pois.extend(argrelextrema(diff,np.less)[0])
pois.append(0)
pois.append(len(diff))
pois.sort()

plt.plot(np.array(upper_decentile)/100)

In [ ]:
diff = gaussian_filter(np.mean(np.abs(closed[0:-2]-closed[1:-1]), axis=(1,2)),5)
plt.plot(diff)


pois = []
#pois.extend(argrelextrema(diff,np.greater)[0])
pois.extend(argrelextrema(diff,np.less)[0])
pois.append(0)
pois.append(len(diff))
pois.sort()

plt.plot(np.array(upper_decentile)/100)

In [ ]:
plt.plot(fourier(diff - gaussian_filter(diff,5))[1])

In [ ]:
closed = gaussian_filter(closed, [0,4,4])

In [ ]:
closed = gaussian_filter(closed, [4,0,0])

In [ ]:
closed = normalize(closed)

In [ ]:
plt.plot(np.mean(closed, axis = (1,2)))

In [ ]:
#closed[closed<.1] = .1
#closed[closed>.75] = .75

#closed = normalize(closed)

In [ ]:
np.mean(closed[100])-np.std(closed[100])

In [ ]:
np.mean(closed[200])+np.std(closed[200])

In [ ]:
diff = gaussian_filter(np.mean(np.abs(closed[100:200]-closed[99:199]), axis=(1,2)),4)
plt.plot(diff)


pois = []
#pois.extend(argrelextrema(diff,np.greater)[0])
pois.extend(argrelextrema(diff,np.less)[0])
pois.append(0)
pois.append(len(diff))
pois.sort()
print(pois)

In [ ]:
%%capture
ani = show_video(normalize(closed[100:200]), n_frames = 200)

In [ ]:
HTML(ani)

In [ ]:
%%capture
ani = show_video(normalize(closed[100:100+42]), vmin=.0, vmax=.99)
ani1 = show_video(normalize(closed[100+42:100+68]), vmin=.0, vmax=.99)
ani2 = show_video(normalize(closed[100+68:100+100]), vmin=.0, vmax=.99)

In [ ]:
HTML(ani)

In [ ]:
HTML(ani1)

In [ ]:
HTML(ani2)

In [ ]:
vessels = filter_blood_vessels(smooth, .4, size_increment=3, n_frames=300)
plt.imshow(vessels)

In [ ]:
interpolated = interpolate_nan_framewise(smooth[70:170], vessels)

In [ ]:
interpolated = gaussian_filter(interpolated,3)

In [ ]:
%%capture
ani = show_video(normalize(interpolated), n_frames=60)

In [ ]:
HTML(ani)

# Horn and Schunck dense optical flow

In [ ]:
for_optical_flow = normalize(closed[100:100+42])

In [ ]:
x_comp, y_comp = horn_schunck(for_optical_flow, 41)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], for_optical_flow[0])
start = 0
n_frames = 40

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]*5,y_comp[i]*5, for_optical_flow[i], fig=fig, ax=ax, quivstep = 5, vmin=.0,vmax=1)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
x_comp, y_comp = horn_schunck(interpolated, 99)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], smooth[1])
start = 0
n_frames = 60

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]*5,y_comp[i]*5, interpolated[i+1], fig=fig, ax=ax, quivstep = 3, vmin=.0,vmax=1)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
roi = sample_roi(details,0,100)

In [ ]:
x_comp, y_comp = horn_schunck(roi, len(roi)-1)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 0
frames = 10

def animate(i):
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]/5,y_comp[i]/5, roi[i+1], fig=fig, ax=ax, scale=10, quivstep=1)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

In [ ]:
HTML(ani.to_jshtml())

# Conclusion

One can filter small scale motion patterns and largescale dynamics. The big size of the data represents a challange becuase of working memory restrictions when using NumPy methods directly. Custom methods can help to reduce the memory requirements. Developing scripts that run in a computational grid on computers with large memory capacities could also help.